In [211]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np

# Get list of FIA Grade One Circuits

In [212]:
#List of FIA Grade One Circuit Cities
city = []
country = []

#Wikipedia page of FIA Grade Circuit Cities
url = 'https://en.wikipedia.org/wiki/List_of_motor_racing_circuits_by_FIA_grade'

url_text = requests.get(url).text

In [213]:
#Read the table from the Wikipedia page
soup = BeautifulSoup(url_text, 'html.parser')
grade_one = soup.find('table', {'class': 'wikitable sortable'})
grade_one

<table class="wikitable sortable" style="font-size: 95%;" width="75%">
<tbody><tr>
<th>Circuit
</th>
<th>Location
</th>
<th>Country
</th>
<th>Layout
</th>
<th>Length
</th>
<th>Continent
</th></tr>
<tr>
<td><a href="/wiki/Albert_Park_Circuit" title="Albert Park Circuit">Albert Park Circuit</a>
</td>
<td><a href="/wiki/Melbourne" title="Melbourne">Melbourne</a>
</td>
<td><span class="flagicon"><img alt="" class="thumbborder" data-file-height="640" data-file-width="1280" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/23px-Flag_of_Australia_%28converted%29.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/35px-Flag_of_Australia_%28converted%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/46px-Flag_of_Australia_%28converted%29.svg.png 2x" width="23"/> </span><a href="/wiki/Australia" title="Australia"

In [214]:
#Create pandas dataframe from the table
df = pd.read_html(str(grade_one))
df = pd.DataFrame(df[0])
df

,Circuit,Location,Country,Layout,Length,Continent
0,Albert Park Circuit,Melbourne,Australia,Grand Prix,5.279 km (3.280 mi),Oceania
1,Autódromo Fernanda Pires da Silva[N 1],Estoril,Portugal,Grand Prix,4.182 km (2.599 mi),Europe
2,Autódromo Hermanos Rodríguez,Mexico City,Mexico,Grand Prix,4.304 km (2.674 mi),North America
3,Autódromo Internacional do Algarve,Portimão,Portugal,Grand Prix,4.653 km (2.891 mi),Europe
4,Autodromo Internazionale del Mugello[N 2],Scarperia e San Piero,Italy,Grand Prix,5.245 km (3.259 mi),Europe
5,Autodromo Internazionale Enzo e Dino Ferrari,Imola,Italy,Grand Prix,4.909 km (3.050 mi),Europe
6,Autódromo José Carlos Pace,São Paulo,Brazil,Grand Prix,4.309 km (2.677 mi),South America
7,Autodromo Nazionale di Monza,Monza,Italy,Grand Prix,5.793 km (3.600 mi),Europe
8,Bahrain International Circuit,Sakhir,Bahrain,Grand Prix,5.412 km (3.363 mi),Asia
9,Bahrain International Circuit,Sakhir,Bahrain,Endurance,6.299 km (3.914 mi),Asia


In [215]:
#Remove wikipedia citations from the dataframe
df['Circuit'] = df['Circuit'].str.replace('\[.*\]', '')

C:\Users\mitch\AppData\Local\Temp\ipykernel_13440\3673038172.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Circuit'] = df['Circuit'].str.replace('\[.*\]', '')


In [216]:
#Clean the dataframe
df.drop_duplicates(subset ="Circuit", keep = 'first', inplace = True)
df.reset_index(drop=True, inplace=True)
fia_grade_one_circuits = df.drop(columns=['Layout', 'Length', 'Continent'])

### Add Lat & Lng from Wiki

In [217]:
#Get circuit coordinates from wikiepdia
def get_coordinates(circuit):
    url = 'https://en.wikipedia.org/wiki/' + circuit
    url_text = requests.get(url).text
    soup = BeautifulSoup(url_text, 'html.parser')
    coordinates = soup.find('span', {'class': 'geo-dec'})
    return coordinates

In [218]:
#Use get_coordinates function to get coordinates for each circuit
coordinates = []
for circuit in fia_grade_one_circuits['Circuit']:
    coordinates.append(get_coordinates(circuit))

In [219]:
#Clean the coordinates list
for i in coordinates:
    if i == None:
        i = np.nan
    else:
        coordinates[coordinates.index(i)] = i.text
    i = (re.search('>(.*)<', str(i)))

In [220]:
#Add coordinates to dataframe
fia_grade_one_circuits['coordinates'] = coordinates

In [222]:
#Split coordinates into latitude and longitude
fia_grade_one_circuits[['Lat', 'Lng']] = fia_grade_one_circuits.coordinates.str.split(" ", expand=True)
fia_grade_one_circuits.drop(columns=['coordinates'], inplace=True)

In [223]:
fia_grade_one_circuits = fia_grade_one_circuits2.dropna()

In [226]:
#Convert latitude and longitude to float
for i, x in fia_grade_one_circuits.iterrows():
    if 'S' in fia_grade_one_circuits['Lat'][i]:
        #fia_grade_one_circuits['Lat'][i] = fia_grade_one_circuits['Lat'][i].replace('°S', '')
        #fia_grade_one_circuits['Lat'][i] = float(fia_grade_one_circuits['Lat'][i]) * -1
    else:
        fia_grade_one_circuits['Lat'][i] = fia_grade_one_circuits['Lat'][i].replace('°N', '')
        fia_grade_one_circuits['Lat'][i] = float(fia_grade_one_circuits['Lat'][i])

for i, x in fia_grade_one_circuits.iterrows():
    if 'W' in fia_grade_one_circuits['Lng'][i]:
        fia_grade_one_circuits['Lng'][i] = fia_grade_one_circuits['Lng'][i].replace('°W', '')
        fia_grade_one_circuits['Lng'][i] = float(fia_grade_one_circuits['Lng'][i]) * -1
    else:
        fia_grade_one_circuits['Lng'][i] = fia_grade_one_circuits['Lng'][i].replace('°E', '')
        fia_grade_one_circuits['Lng'][i] = float(fia_grade_one_circuits['Lng'][i])

In [228]:
#Export the dataframe to a csv file
fia_grade_one_circuits.to_csv('Exported Dataframes/grade_one_circuits.csv')

fia_grade_one_circuits

,Circuit,Location,Country,Lat,Lng
0,Albert Park Circuit,Melbourne,Australia,-37.84972,144.96833
1,Autódromo Fernanda Pires da Silva,Estoril,Portugal,38.75083,-9.39417
2,Autódromo Hermanos Rodríguez,Mexico City,Mexico,19.40611,-99.0925
3,Autódromo Internacional do Algarve,Portimão,Portugal,37.232,-8.632
4,Autodromo Internazionale del Mugello,Scarperia e San Piero,Italy,43.9975,11.37194
5,Autodromo Internazionale Enzo e Dino Ferrari,Imola,Italy,44.34111,11.71333
6,Autódromo José Carlos Pace,São Paulo,Brazil,-23.70111,-46.69722
7,Autodromo Nazionale di Monza,Monza,Italy,45.62056,9.28944
8,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.51056
9,Baku City Circuit,Baku,Azerbaijan,40.3725,49.85333


# Get Historical Weather Data

In [232]:
data = requests.get('https://www.wunderground.com/history/airport/EGKK/2019/1/1/MonthlyHistory.html?req_city=London&req_state=&req_statename=England&reqdb.zip=00000&reqdb.magic=1&reqdb.wmo=03712&MR=1').text